In [166]:
import os
import numpy as np
import pandas as pd
import yfinance as yf

In [167]:
from enum import Enum

In [168]:
last_year = 2019  # will iterate backwards
working_days = 250  # number of working days in a year
upper_bound = last_year + 1  # won't include this year

In [169]:
folder_path = '../third-party/US Stocks/'

In [170]:
NYSE_PATH, AMEX_PATH, NASDAQ_PATH = 'NYSE.csv', 'AMEX.csv', 'NASDAQ.csv'

In [171]:
def get_tickers():
    def to_list(filepath):
        ticker_list = pd.read_csv(filepath, engine='python')
        return ticker_list[~ticker_list['Symbol'].str.contains("\.|\^")]['Symbol'].tolist()
    # Return the ticks from all available traders
    return (to_list(folder_path + NYSE_PATH), to_list(folder_path + AMEX_PATH), to_list(folder_path + NASDAQ_PATH))

In [7]:
for tick in get_tickers():
    stock = yf.Ticker(tick)
    stock_history = stock.history(period="max")
    stock_history = stock_history[['Open', 'High', 'Close', 'Volume']]
    stock_history.to_pickle('../third-party/US Stocks/Stocks/%s.pkl' % tick)

- BMY~: No data found, symbol may be delisted
- BEPC$: No data found, symbol may be delisted
- BEP$: No data found, symbol may be delisted
- CTT           : No data found, symbol may be delisted
- CELG~: No data found, symbol may be delisted
- ECOM          : No data found, symbol may be delisted
- ECC           : No data found, symbol may be delisted
- ETX           : No data found, symbol may be delisted
- FEI           : No data found, symbol may be delisted
- GEN           : No data found, symbol may be delisted
- GIX~: No data found, symbol may be delisted
- STAR          : No data found, symbol may be delisted
- NTEST: 1d data not available for startTime=-2208988800 and endTime=1596199403. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CTEST: 1d data not available for startTime=-2208988800 and endTime=1596199476. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SAND          : No data found, symbol may be

- CRSAW: 1d data not available for startTime=-2208988800 and endTime=1596200084. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CYRXW: 1d data not available for startTime=-2208988800 and endTime=1596200087. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DRIOW: 1d data not available for startTime=-2208988800 and endTime=1596200098. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DSKEW: 1d data not available for startTime=-2208988800 and endTime=1596200099. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DRMT: 1d data not available for startTime=-2208988800 and endTime=1596200104. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DFPHW: 1d data not available for startTime=-2208988800 and endTime=1596200106. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DPHCW: 1d

- IMACW: 1d data not available for startTime=-2208988800 and endTime=1596200319. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IMTXW: 1d data not available for startTime=-2208988800 and endTime=1596200320. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IMRNW: 1d data not available for startTime=-2208988800 and endTime=1596200323. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IEAWW: 1d data not available for startTime=-2208988800 and endTime=1596200327. Only 100 years worth of day granularity data are allowed to be fetched per request.
- INSUW: 1d data not available for startTime=-2208988800 and endTime=1596200334. Only 100 years worth of day granularity data are allowed to be fetched per request.
- IGICW: 1d data not available for startTime=-2208988800 and endTime=1596200339. Only 100 years worth of day granularity data are allowed to be fetched per request.
- CMFNL: 1

- SAMAW: 1d data not available for startTime=-2208988800 and endTime=1596200717. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SHIPW: 1d data not available for startTime=-2208988800 and endTime=1596200720. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SHIPZ: 1d data not available for startTime=-2208988800 and endTime=1596200720. Only 100 years worth of day granularity data are allowed to be fetched per request.
- EYESW: 1d data not available for startTime=-2208988800 and endTime=1596200721. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SCCI: 1d data not available for startTime=-2208988800 and endTime=1596200734. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SGLBW: 1d data not available for startTime=-2208988800 and endTime=1596200739. Only 100 years worth of day granularity data are allowed to be fetched per request.
- SSPKW: 1d

In [172]:
while True:
    shared_idx, shared_col, collective_frames = {}, {}, {}
    for root, dirs, files in os.walk("../third-party/US Stocks/Stocks/"):
        for filename in files:
            stock = pd.read_pickle("%s/%s" % (root, filename))
            lower_limit = (stock.index > pd.to_datetime('%s-01-01' % last_year))
            upper_limit = (stock.index < pd.to_datetime('%d-01-01' % upper_bound))
            x = upper_limit & lower_limit
            if len(stock.index[x]) >= (working_days * (upper_bound - last_year)):
                # Identifying the shared timestamps and shared columns
                shared_idx = set(stock.index[x]) if len(shared_idx) == 0 else set(shared_idx) | set(stock.index[x])
                shared_col = set(stock.columns) if len(shared_col) == 0 else set(shared_col) | set(stock.columns)
                # Saving the current dataset for later use
                collective_frames[filename] = stock
            elif len(stock.index) == 0:
                os.remove("%s/%s" % (root, filename))
    if len(collective_frames.keys()) == 0: break  # No year has left
    # Verbosing results from each year
    print(last_year, len(collective_frames.keys()))
    # Shared indice and columns for the current year
    shared_idx = sorted(shared_idx)
    shared_col = sorted(shared_col)
    # Resulting tensor
    dataframes = []
    for key in list(sorted(collective_frames.keys())):
        dataframe = collective_frames[key].reindex(shared_idx).sort_index()
        dataframe = dataframe[shared_col].fillna(method='ffill')
        dataframe = dataframe.fillna(method='bfill')
        if dataframe.isna().sum().sum() > 0:
            print(key, "FAIL!")
        dataframes.append(dataframe.values)
    matrix = np.stack(dataframes)
    # Preparing for saving
    shared_idx = [str(idx) for idx in shared_idx]
    shared_col = [str(idx) for idx in shared_col]
    sulfix = "../prepared-data/US Stocks %d:%d" % (last_year, (upper_bound - 1))
    # Saving the tensor
    np.save(sulfix + " (%dx%dx%d)" % matrix.shape, matrix)
    # 0-Axis description
    with open(sulfix + " (%dx%dx%d).AX0" % matrix.shape,'w') as f:
        f.write('\n'.join(list(sorted(collective_frames.keys()))))
    # 1-Axis description
    with open(sulfix + " (%dx%dx%d).AX1" % matrix.shape,'w') as f:
        f.write('\n'.join(shared_idx))
    # 2-Axis description
    with open(sulfix + " (%dx%dx%d).AX2" % matrix.shape,'w') as f:
        f.write('\n'.join(shared_col))
    last_year -= 1  # Iterating over years

2019 5466
2018 5093
2017 4794
2016 4548
2015 4301
2014 3997
2013 3767
2012 3570
2011 3436
2010 3270
2009 3182
2008 3112
2007 2913
2006 2761
2005 2612
2004 2469
2003 2341
2002 2241
2001 2173
2000 2061
1999 1914
1998 1789
1997 1682
1996 1564
1995 1445
1994 1309
1993 1171
1992 1058
1991 968
1990 939
1989 892
1988 844
1987 775
1986 694
1985 657
1984 620
1983 575
1982 553
1981 511
1980 476
1979 195
1978 193
1977 188
1976 179
1975 173
1974 171
1973 169
1972 46
1971 27
1970 27
1969 20
1968 20
1967 20
1966 19
1965 19
1964 19
1963 19
1962 19
